In [2]:
#!pip install rank-bm25
#from rank_bm25 import BM25Okapi
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
import os
from os import listdir
from os.path import isfile, join
import re
import numpy as np
from math import floor, ceil

In [5]:
path = r"../src/data/"
file_path = path + r"LocalBusiness/LB_cleaned"
files = os.listdir(file_path)


In [6]:
df_all = pd.DataFrame()
for file in files:
    df = pd.read_json(os.path.join(file_path, file), compression='gzip', lines=True)
    df['table id'] = file.strip('.json.gz')
    df_all = pd.concat([df_all, df], ignore_index=True)

In [13]:
len(df_all)

215069

## Only get entries where dataframe has non-zero telephone numbers

In [11]:
df_tel = df_all[~df_all["telephone"].isna()]

In [12]:
df_tel

,row_id,name,address,telephone,page_url,table id,aggregaterating,email,geo,url,...,areaserved,pricerange,fax,interactioncount,hasoffercatalog,paymentaccepted,currenciesaccepted,brand,knowsabout,legalname
0,1.0,Law Office of Donald E Green,"{'addressregion': 'MA', 'postalcode': '02368',...",(781) 885-3244,https://101attorney.com/attorney_finder/ma_oth...,LocalBusiness_101attorney.com_September2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,The Law Offices Of Joshua F Pompilus Pc,"{'streetaddress': '490 N Main Street', 'addres...",(781) 986-1124,https://101attorney.com/attorney_finder/ma_oth...,LocalBusiness_101attorney.com_September2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,"Tiana M. Giebultowski, Attorney at Law","{'addressregion': 'MA', 'postalcode': '02368',...",(339) 987-8258,https://101attorney.com/attorney_finder/ma_oth...,LocalBusiness_101attorney.com_September2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6.0,Girtz Eial PC Attorney At Law,"{'addressregion': 'NY', 'streetaddress': '185 ...",(516) 466-6912,https://101attorney.com/attorney_finder/ny_gre...,LocalBusiness_101attorney.com_September2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7.0,"Kiley, Kiley & Kiley, PLLC - Attorneys At Law","{'addressregion': 'NY', 'postalcode': '11021',...",(516) 466-7900,https://101attorney.com/attorney_finder/ny_gre...,LocalBusiness_101attorney.com_September2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207001,14004.0,"Top 10 Public Speaking Tutors Near Aventura, F...",NaN,13126466365,https://www.wyzant.com/Aventura_FL_public_spea...,LocalBusiness_wyzant.com_September2020,"{'ratingvalue': '4.96', 'reviewcount': '727'}",NaN,NaN,NaN,...,"{'name': 'Aventura, FL'}",NaN,NaN,NaN,NaN,NaN,NaN,"{'name': 'Wyzant', 'telephone': '+13126466365'}",Public Speaking,NaN
207002,14005.0,"Top 10 Spelling Tutors Near Edina, MN | Wyzant",NaN,13126466365,https://www.wyzant.com/Edina_MN_spelling_tutor...,LocalBusiness_wyzant.com_September2020,"{'reviewcount': '1333', 'ratingvalue': '4.91'}",NaN,NaN,NaN,...,"{'name': 'Edina, MN'}",NaN,NaN,NaN,NaN,NaN,NaN,"{'name': 'Wyzant', 'telephone': '+13126466365'}",Spelling,NaN
207003,14006.0,Top 10 Language Tutors Near Missouri | Wyzant,NaN,13126466365,https://www.wyzant.com/Missouri_Language_tutor...,LocalBusiness_wyzant.com_September2020,"{'ratingvalue': '4.95', 'reviewcount': '297'}",NaN,NaN,NaN,...,{'name': 'Missouri'},NaN,NaN,NaN,NaN,NaN,NaN,"{'name': 'Wyzant', 'telephone': '+13126466365'}",Language,NaN
207004,14007.0,"Top Algebra 2 Tutors Near Roanoke, VA | Wyzant",NaN,13126466365,https://www.wyzant.com/Roanoke_VA_algebra_2_tu...,LocalBusiness_wyzant.com_September2020,"{'reviewcount': '261', 'ratingvalue': '4.97'}",NaN,NaN,NaN,...,"{'name': 'Roanoke, VA'}",NaN,NaN,NaN,NaN,NaN,NaN,"{'name': 'Wyzant', 'telephone': '+13126466365'}",Algebra 2,NaN


In [24]:
example = df_tel.iloc[0]["telephone"]
example

'(781) 885-3244'

In [29]:
re.findall('[1-9]+', example)

['781', '885', '3244']

In [14]:
def normalizer(telephone):    
    return ''.join(re.findall('[1-9]+', telephone))
 
df_tel['telephone_new'] = df_tel['telephone'].astype('str').apply(normalizer)

C:\Users\Jannik\AppData\Local\Temp/ipykernel_16756/3635726827.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tel['telephone_new'] = df_tel['telephone'].astype('str').apply(normalizer)


In [54]:
df_tel

,row_id,name,address,telephone,page_url,table id,aggregaterating,email,geo,url,...,pricerange,fax,interactioncount,hasoffercatalog,paymentaccepted,currenciesaccepted,brand,knowsabout,legalname,telephone_new
0,1.0,Law Office of Donald E Green,"{'addressregion': 'MA', 'postalcode': '02368',...",(781) 885-3244,https://101attorney.com/attorney_finder/ma_oth...,LocalBusiness_101attorney.com_September2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7818853244
1,2.0,The Law Offices Of Joshua F Pompilus Pc,"{'streetaddress': '490 N Main Street', 'addres...",(781) 986-1124,https://101attorney.com/attorney_finder/ma_oth...,LocalBusiness_101attorney.com_September2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7819861124
2,3.0,"Tiana M. Giebultowski, Attorney at Law","{'addressregion': 'MA', 'postalcode': '02368',...",(339) 987-8258,https://101attorney.com/attorney_finder/ma_oth...,LocalBusiness_101attorney.com_September2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3399878258
3,6.0,Girtz Eial PC Attorney At Law,"{'addressregion': 'NY', 'streetaddress': '185 ...",(516) 466-6912,https://101attorney.com/attorney_finder/ny_gre...,LocalBusiness_101attorney.com_September2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5164666912
4,7.0,"Kiley, Kiley & Kiley, PLLC - Attorneys At Law","{'addressregion': 'NY', 'postalcode': '11021',...",(516) 466-7900,https://101attorney.com/attorney_finder/ny_gre...,LocalBusiness_101attorney.com_September2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51646679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207001,14004.0,"Top 10 Public Speaking Tutors Near Aventura, F...",NaN,13126466365,https://www.wyzant.com/Aventura_FL_public_spea...,LocalBusiness_wyzant.com_September2020,"{'ratingvalue': '4.96', 'reviewcount': '727'}",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'name': 'Wyzant', 'telephone': '+13126466365'}",Public Speaking,NaN,13126466365
207002,14005.0,"Top 10 Spelling Tutors Near Edina, MN | Wyzant",NaN,13126466365,https://www.wyzant.com/Edina_MN_spelling_tutor...,LocalBusiness_wyzant.com_September2020,"{'reviewcount': '1333', 'ratingvalue': '4.91'}",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'name': 'Wyzant', 'telephone': '+13126466365'}",Spelling,NaN,13126466365
207003,14006.0,Top 10 Language Tutors Near Missouri | Wyzant,NaN,13126466365,https://www.wyzant.com/Missouri_Language_tutor...,LocalBusiness_wyzant.com_September2020,"{'ratingvalue': '4.95', 'reviewcount': '297'}",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'name': 'Wyzant', 'telephone': '+13126466365'}",Language,NaN,13126466365
207004,14007.0,"Top Algebra 2 Tutors Near Roanoke, VA | Wyzant",NaN,13126466365,https://www.wyzant.com/Roanoke_VA_algebra_2_tu...,LocalBusiness_wyzant.com_September2020,"{'reviewcount': '261', 'ratingvalue': '4.97'}",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"{'name': 'Wyzant', 'telephone': '+13126466365'}",Algebra 2,NaN,13126466365


In [60]:
df_tel["telephone_new"].value_counts().head(10)

13126466365    12737
                2944
9312             483
18223344         299
226128           276
256281           244
25655515          60
66999             36
18181235          29
18132222          23
Name: telephone_new, dtype: int64

this telephone number is not useful from this dataset

In [61]:
df_tel[df_tel["telephone_new"] == "18132222"]

,row_id,name,address,telephone,page_url,table id,aggregaterating,email,geo,url,...,pricerange,fax,interactioncount,hasoffercatalog,paymentaccepted,currenciesaccepted,brand,knowsabout,legalname,telephone_new
40144,2.0,Punjab National Bank Customer Care Number,"{'addressregion': 'Nawanshahr', 'streetaddress...",1.8001e+10,https://www.grotal.com/Nawanshahr/Dena-Bank-He...,LocalBusiness_grotal.com_September2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18132222
40178,119.0,Punjab National Bank Customer Care Number,"{'addresslocality': 'Haridwar', 'addressregion...",1.8001e+10,https://www.grotal.com/Haridwar/Dhanlaxmi-Bank...,LocalBusiness_grotal.com_September2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18132222
40190,177.0,Punjab National Bank Customer Care Number,"{'addresslocality': 'Hamirpur Himachal', 'stre...",1.8001e+10,https://www.grotal.com/Hamirpur-Himachal/SBI-C...,LocalBusiness_grotal.com_September2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18132222
40243,288.0,Punjab National Bank Customer Care Number,"{'addressregion': 'Dehradun', 'addresscountry'...",1.8001e+10,https://www.grotal.com/Dehradun/Axis-Bank-Comp...,LocalBusiness_grotal.com_September2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18132222
40391,715.0,Punjab National Bank Customer Care Number,"{'addressregion': 'Sirsa', 'addresslocality': ...",1.8001e+10,https://www.grotal.com/Sirsa/HSBC-Bank-Helplin...,LocalBusiness_grotal.com_September2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18132222
40413,760.0,Punjab National Bank Customer Care Number,"{'streetaddress': 'Circular Road', 'addressreg...",1.8001e+10,https://www.grotal.com/Bhiwani/Oriental-Bank-o...,LocalBusiness_grotal.com_September2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18132222
40435,813.0,Punjab National Bank Customer Care Number,"{'addressregion': 'Shimla', 'addresslocality':...",1.8001e+10,https://www.grotal.com/Shimla/Federal-Bank-Com...,LocalBusiness_grotal.com_September2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18132222
40456,866.0,Punjab National Bank Customer Care Number,"{'addressregion': 'Rewari', 'addresslocality':...",1.8001e+10,https://www.grotal.com/Rewari/HDFC-Bank-Custom...,LocalBusiness_grotal.com_September2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18132222
40677,1593.0,Punjab National Bank Customer Care Number,"{'streetaddress': 'Janta Nagar', 'addressregio...",1.8001e+10,https://www.grotal.com/Dhuri/Central-Bank-of-I...,LocalBusiness_grotal.com_September2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18132222
40832,2096.0,Punjab National Bank Customer Care Number,"{'addresscountry': 'IN', 'streetaddress': 'Mai...",1.8001e+10,https://www.grotal.com/Naraingarh/Karnataka-Ba...,LocalBusiness_grotal.com_September2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18132222
